In [3]:
import numpy as np
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d  # because we will be using projection='3d'
from ipywidgets import interact_manual, FloatSlider

In [4]:
def graph3d_wrapper(func, xlabel:str, ylabel:str, zlabel:str, title:str, xlower:float, xupper:float,xstep:float,
                    ylower:float, yupper:float, ystep:float, proj:bool):
    """
    Wrapper function for making the plotting function easily usable for 'interact_manual'.
    Set values to all arguments except the one manipulated by 'FloatSlider'.
    """
    
    def graph3d(cov):

        fig = plt.figure()
        ax = fig.gca(projection='3d')
        
        xs = np.arange(xlower, xupper, xstep)
        ys = np.arange(ylower, yupper, ystep)
        xs, ys = np.meshgrid(xs, ys)
        zs = func(xs, ys, cov)
        
        ax.plot_surface(xs, ys, zs, alpha=0.7, cmap=cm.viridis)
        if proj:
            cset = ax.contourf(xs, ys, zs, zdir='z', offset=zs.min(), cmap=cm.coolwarm)

        ax.set_xlim(xlower, xupper); ax.set_ylim(ylower, yupper); ax.set_zlim(zs.min(), zs.max())
        ax.set_title(title)
        ax.set_xlabel(xlabel); ax.set_ylabel(ylabel); ax.set_zlabel(zlabel)
        plt.tight_layout()
        plt.show()
    
    return graph3d

## Definition of multi-variate normal distribution

<img src='normal-2d.png' width=500>

## Definition of Mahalanobis distance, $\triangle$

$\triangle = ({\bf x}-{\boldsymbol \mu})^T \boldsymbol{\Sigma} ({\bf x}-{\boldsymbol \mu})$, which reduces to Euclidean distance when $\boldsymbol{\Sigma}$ is the identity matrix.

## Understand bi-variate Gaussian through algebraic expansion of $\triangle$

Concepts needed:

- Recall the definition of **covariance**: $\text{cov}({\bf x}, {\bf y})=\mathbb{E}[({\bf x}-\mu_{\bf x})({\bf y}-\mu_{\bf y})]$. Intuitively, covariance measures the extent to which two random variables simultaneously (not necessarily w.r.t time) move above and below their means. 

- Recall the definition of **matrix inverse**: <img src="https://github.com/zhihanyang2022/Bishop-Pattern-Recognition-Notes/blob/master/pngs/matrix-inverse.png" alt="matrix inverse" width=300>

Understanding the expression in the exponent:

- $\bf{x}$ is $[\bf{x}_1, \bf{x}_2]^T$ (column vector) and ${\boldsymbol \mu}$ is $[\mu_1, \mu_2]^T$ (column vector). $\bf{x}_1$ and $\bf{x}_2$ can be considered as two random variables. Whether they are independent or dependent is determined by the covariance matrix.
---
- $\boldsymbol{\Sigma}$ is a 2-by-2 covariance matrix, $\begin{bmatrix} \text{cov}({\bf x}_1,{\bf x}_1) & \text{cov}({\bf x}_1,{\bf x}_2) \\ \text{cov}({\bf x}_1,{\bf x}_2) & \text{cov}({\bf x}_2,{\bf x}_2) \end{bmatrix}$. 
---
- $({\bf x} - {\boldsymbol \mu})^T = ([x_1, x_2]^T - [\mu_1, \mu_2]^T)^T = [x_1-\mu_1, x_2-\mu_2]$ is a row vector.
---
- $ \begin{align} \boldsymbol{\Sigma}^{-1} &=\begin{bmatrix} \text{cov}({\bf x}_1,{\bf x}_1) & \text{cov}({\bf x}_1,{\bf x}_2) \\ \text{cov}({\bf x}_1,{\bf x}_2) & \text{cov}({\bf x}_2,{\bf x}_2) \end{bmatrix}^{-1} \\ &= \frac{1}{\text{det}(\boldsymbol{\Sigma})} \begin{bmatrix} \text{cov}({\bf x}_2,{\bf x}_2) & - \text{cov}({\bf x}_1,{\bf x}_2) \\ - \text{cov}({\bf x}_1,{\bf x}_2) & \text{cov}({\bf x}_1,{\bf x}_1) \end{bmatrix} \\ &= \frac{1}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2} \begin{bmatrix} \text{cov}({\bf x}_2,{\bf x}_2) & - \text{cov}({\bf x}_1,{\bf x}_2) \\ - \text{cov}({\bf x}_1,{\bf x}_2) & \text{cov}({\bf x}_1,{\bf x}_1) \end{bmatrix} \\ &= \begin{bmatrix} \frac{\text{cov}({\bf x}_2,{\bf x}_2)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2} & - \frac{\text{cov}({\bf x}_1,{\bf x}_2)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2} \\ - \frac{\text{cov}({\bf x}_1,{\bf x}_2)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2} & \frac{\text{cov}({\bf x}_1,{\bf x}_1)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2} \end{bmatrix} \\ \end{align}$
---
- $\begin{align} ({\bf x} - {\boldsymbol \mu})^T \boldsymbol{\Sigma}^{-1}({\bf x} - {\boldsymbol \mu}) &= [x_1-\mu_1, x_2-\mu_2] \begin{bmatrix} \frac{\text{cov}({\bf x}_2,{\bf x}_2)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2} & - \frac{\text{cov}({\bf x}_1,{\bf x}_2)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2} \\ - \frac{\text{cov}({\bf x}_1,{\bf x}_2)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2} & \frac{\text{cov}({\bf x}_1,{\bf x}_1)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2} \end{bmatrix} [{\bf x}_1-\mu_1, {\bf x}_2-\mu_2]^T \\ &= [({\bf x}_1-\mu_1)^2\frac{\text{cov}({\bf x}_2,{\bf x}_2)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2}- ({\bf x}_1-\mu_1)({\bf x}_2-\mu_2)\frac{\text{cov}({\bf x}_1,{\bf x}_2)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2} -({\bf x}_2-\mu_2)({\bf x}_1-\mu_1)\frac{\text{cov}({\bf x}_1,{\bf x}_2)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2}+({\bf x}_2-\mu_2)^2\frac{\text{cov}({\bf x}_1,{\bf x}_1)}{\text{cov}({\bf x}_1,{\bf x}_1)\text{cov}({\bf x}_2,{\bf x}_2)-\text{cov}({\bf x}_1,{\bf x}_2)^2}]  \end{align}$

---

- Assume that we are modelling normalized data, that is, data with zero mean ($\mu_1=0$, $\mu_2=0$) and unit variance, then $\text{cov}({\bf x}_1, {\bf x}_1)=1$ and $\text{cov}({\bf x}_2, {\bf x}_2)=1$. As a result, $-1 \leq \text{cov}({\bf x}_1, {\bf x}_2) \leq 1$ and $0 \leq \text{cov}({\bf x}_1, {\bf x}_2)^2 \leq 1$.

---

- $ \begin{align} ({\bf x} - {\boldsymbol \mu})^T \boldsymbol{\Sigma}^{-1}({\bf x} - {\boldsymbol \mu}) &= {\bf x}_1^2\frac{1}{1-\text{cov}({\bf x}_1,{\bf x}_2)^2} - 2{\bf x}_1{\bf x}_2\frac{\text{cov}({\bf x}_1,{\bf x}_2)}{1-\text{cov}({\bf x}_1,{\bf x}_2)^2} + {\bf x}_2^2\frac{1}{1-\text{cov}({\bf x}_1,{\bf x}_2)^2} \\ &= \color{red}{- \frac{1}{2}{\bf x}_1^2\frac{1}{1-\text{cov}({\bf x}_1,{\bf x}_2)^2}} \color{green}{+ {\bf x}_1{\bf x}_2\frac{\text{cov}({\bf x}_1,{\bf x}_2)}{1-\text{cov}({\bf x}_1,{\bf x}_2)^2}} \color{red}{- \frac{1}{2}{\bf x}_2^2\frac{1}{1-\text{cov}({\bf x}_1,{\bf x}_2)^2}} \\ &= \color{red}{- a {\bf x}_1^2} \color{green}{+ b{\bf x}_1{\bf x}_2} \color{red}{- a {\bf x}_2^2} \text{ where } a \text{ and } b \text{ are constants.} \end{align} $

---

- I name the two terms in red **"quadratic terms"** simply because they form <u> concave quadratic surfaces</u> with <u>perfect rotational symmetry about the z-axis</u>.
- I name the term in green **"covariance term"**. It deserves a name of its own because it <u>modifies the shape of quadratic surfaces</u> when added to the quadratic terms. The extent to which it does this depends on the magnitude of covariance.

---

### Visualize quadratic terms, $\color{red}{- \frac{1}{2}{\bf x}_1^2\frac{1}{1-\text{cov}({\bf x}_1,{\bf x}_2)^2}}\color{red}{- \frac{1}{2}{\bf x}_2^2\frac{1}{1-\text{cov}({\bf x}_1,{\bf x}_2)^2}}$

The higher the magnitude of the covariance (regardless or sign, due to the fact that the covariance is squared):
- the higher the magnitude of the quadratic terms (negatively) and 
- the sharper the exponentiated surface.

In [5]:
def quadratic_term(xs, ys, cov):
    zs = np.zeros(xs.shape)
    for i in range(xs.shape[0]):
        for j in range(xs.shape[1]):
            z = - 0.5 * (1 / (1 - cov**2)) * (xs[i, j]**2 + ys[i, j]**2)  # quadratic term formula
            zs[i][j] = z
    return zs

In [6]:
quadratic_term_plotter = graph3d_wrapper(quadratic_term, 
                                         'x1', 'x2', 'Quadratic Term', 'Quadratic Term', 
                                         -5, 5, 0.05, -5, 5, 0.05, 
                                         proj=True)
interact_manual(quadratic_term_plotter, cov=FloatSlider(min=-0.9, max=0.9, step=1e-2), continuous_update=False);

interactive(children=(FloatSlider(value=0.0, description='cov', max=0.9, min=-0.9, step=0.01), Button(descript…

In [7]:
exp_quadratic_term_plotter = graph3d_wrapper(lambda xs, ys, cov : np.e ** quadratic_term(xs, ys, cov), 
                                             'x1', 'x2', 'exp(Quadratic Term)', 'exp(Quadratic Term)', 
                                             -5, 5, 0.05, -5, 5, 0.05, 
                                             proj=True)
_ = interact_manual(exp_quadratic_term_plotter, cov=FloatSlider(min=-0.9, max=0.9, step=1e-2), continuous_update=False)

interactive(children=(FloatSlider(value=0.0, description='cov', max=0.9, min=-0.9, step=0.01), Button(descript…

### Visualize negative covariance term, $\color{green}{{\bf x}_1{\bf x}_2\frac{\text{cov}({\bf x}_1,{\bf x}_2)}{1-\text{cov}({\bf x}_1,{\bf x}_2)^2}} \text{ when  } \text{cov}({\bf x}_1, {\bf x}_2) < 0$

${\bf x}_1 {\bf x}_2$ is multiplied by a negative scalar.

- The covariance term is positive if ${\bf x}_1 {\bf x}_2$ is negative, which happens in second and fourth quadrants of the outcome space.
- The covariance term is negative if ${\bf x}_1 {\bf x}_2$ is positive, which happens in first and third quadrants of the outcome space.

|  Quadrants / Term  |      covariance term     |      exp(covariance term)     |
|:--------------------------:|:------------------------:|:-----------------------------:|
|  first and third quadrant  | negative, grows linearly |        converge to zero       |
| second and fourth quadrant | positive, grows linearly | positive, grows exponentially |
    
A similar analysis can be done easily for positive covariance.

In [8]:
def covariance_term(xs, ys, cov):
    zs = np.zeros(xs.shape)
    for i in range(xs.shape[0]):
        for j in range(xs.shape[1]):
            z = xs[i, j] * ys[i, j] * cov / (1 - cov**2)  # covariance term formula
            zs[i][j] = z
    return zs

In [9]:
covariance_term_plotter = graph3d_wrapper(covariance_term, 
                                          'x1', 'x2', 'Covariance Term', 'Covariance Term', 
                                          -5, 5, 0.05, -5, 5, 0.05, 
                                          proj=True)
interact_manual(covariance_term_plotter, cov=FloatSlider(min=-0.9, max=0.9, step=1e-2), continuous_update=False);

interactive(children=(FloatSlider(value=0.0, description='cov', max=0.9, min=-0.9, step=0.01), Button(descript…

In [10]:
exp_covariance_term_plotter = graph3d_wrapper(lambda xs, ys, cov : np.e ** covariance_term(xs, ys, cov), 
                                              'x1', 'x2', 'exp(Covariance Term)', 'exp(Covariance Term)', 
                                              -5, 5, 0.05, -5, 5, 0.05, 
                                              proj=True)
interact_manual(exp_covariance_term_plotter, cov=FloatSlider(min=-0.9, max=0.9, step=1e-2), continuous_update=False);

interactive(children=(FloatSlider(value=0.0, description='cov', max=0.9, min=-0.9, step=0.01), Button(descript…

### Visualize the quadratic terms and the covariance term together

In [11]:
all_terms_plotter = graph3d_wrapper(lambda xs, ys, cov : quadratic_term(xs, ys, cov) + covariance_term(xs, ys, cov), 
                                     'x1', 'x2', 'Both Terms', 'Both Terms', 
                                     -5, 5, 0.05, -5, 5, 0.05, 
                                     proj=True)
interact_manual(all_terms_plotter, cov=FloatSlider(min=-0.9, max=0.9, step=1e-2), continuous_update=False);

interactive(children=(FloatSlider(value=0.0, description='cov', max=0.9, min=-0.9, step=0.01), Button(descript…

In [12]:
exp_all_terms_plotter = graph3d_wrapper(lambda xs, ys, cov : np.e ** (quadratic_term(xs, ys, cov) + covariance_term(xs, ys, cov)), 
                                        'x1', 'x2', 'exp(Both Terms)', 'exp(Both Terms)', 
                                        -5, 5, 0.05, -5, 5, 0.05, 
                                        proj=True)
interact_manual(exp_all_terms_plotter, cov=FloatSlider(min=-0.9, max=0.9, step=1e-2), continuous_update=False);

interactive(children=(FloatSlider(value=0.0, description='cov', max=0.9, min=-0.9, step=0.01), Button(descript…

## References

- Matrix inverse: https://www.mathsisfun.com/algebra/matrix-inverse.html